In [1]:
# Matrix Factorization Recommendation Model
# Try using Truncated SVD
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors

ratings = pd.read_csv('Books Data/ratings.csv')
books = pd.read_csv('Books Data/books.csv')

In [2]:
ratings_pivot = ratings.pivot(index='book_id', columns='user_id', values='rating').fillna(0)
ratings_pivot

user_id,1,2,3,4,5,6,7,8,9,10,...,53415,53416,53417,53418,53419,53420,53421,53422,53423,53424
book_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,4.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0
2,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,5.0,5.0,5.0,5.0,5.0,5.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,4.0
4,5.0,0.0,3.0,4.0,0.0,0.0,0.0,3.0,0.0,5.0,...,0.0,0.0,0.0,0.0,3.0,0.0,5.0,0.0,5.0,5.0
5,0.0,5.0,0.0,4.0,0.0,0.0,3.0,3.0,5.0,5.0,...,0.0,0.0,0.0,0.0,3.0,2.0,4.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
SVD = TruncatedSVD(n_components=12, random_state=17)
result_matrix = SVD.fit_transform(ratings_pivot)
correlation_mat = np.corrcoef(result_matrix)
correlation_mat.shape

(10000, 10000)

In [4]:
correlation_mat

array([[1.        , 0.79058292, 0.86792297, ..., 0.39637487, 0.56660365,
        0.36876494],
       [0.79058292, 1.        , 0.72542971, ..., 0.47537471, 0.40071773,
        0.32753651],
       [0.86792297, 0.72542971, 1.        , ..., 0.10905194, 0.30494421,
        0.06222215],
       ...,
       [0.39637487, 0.47537471, 0.10905194, ..., 1.        , 0.3330796 ,
        0.82339108],
       [0.56660365, 0.40071773, 0.30494421, ..., 0.3330796 , 1.        ,
        0.41870637],
       [0.36876494, 0.32753651, 0.06222215, ..., 0.82339108, 0.41870637,
        1.        ]])

In [5]:
np.save('correlation_mat.npy', correlation_mat)
# loading in: 
# correlation_mat = np.load('correlation_mat.npy') 

In [6]:
test_mat = np.load('correlation_mat.npy') 

In [7]:
test_mat

array([[1.        , 0.79058292, 0.86792297, ..., 0.39637487, 0.56660365,
        0.36876494],
       [0.79058292, 1.        , 0.72542971, ..., 0.47537471, 0.40071773,
        0.32753651],
       [0.86792297, 0.72542971, 1.        , ..., 0.10905194, 0.30494421,
        0.06222215],
       ...,
       [0.39637487, 0.47537471, 0.10905194, ..., 1.        , 0.3330796 ,
        0.82339108],
       [0.56660365, 0.40071773, 0.30494421, ..., 0.3330796 , 1.        ,
        0.41870637],
       [0.36876494, 0.32753651, 0.06222215, ..., 0.82339108, 0.41870637,
        1.        ]])

In [2]:
from difflib import SequenceMatcher
import re

books_key = books.copy()
books_key.title.fillna(books_key.original_title, inplace=True)
books_key.drop(books_key.columns.difference(['book_id', 'title', 'authors']), 1, inplace=True)
books_key.to_csv("books_key.csv", index=False)

book_hashmap = dict(zip(books_key.book_id, books_key.title))
author_hashmap = dict(zip(books_key.book_id, books_key.authors))

In [41]:
def get_book_title(book_name):
    for key, book in book_hashmap.items():
        temp = re.split('[(,.:)]', book)
        x = 0
        while x < len(temp):
            if SequenceMatcher(a=book_name.lower(), b=temp[x].lower()).ratio() > 0.7:
                print(str(key) + ": " + book)
                x = len(temp)
            x = x + 1
    index = int(input("\nEnter the book index (enter -1 if not found): "))
    return index

In [42]:
def get_book_author(book_author):
    for key, author in author_hashmap.items():
        temp = author.split(',')
        x = 0
        while x < len(temp):
            if SequenceMatcher(a=book_author.lower(), b=temp[x].lower()).ratio() > 0.9:
                print(str(key) + ": " + book_hashmap.get(key) + " by " + author)
                x = len(temp)
            x = x + 1
    index = int(input("\nEnter the book index (enter -1 if not found): "))
    return index

In [61]:
def search_book():
    choice = -1
    index = -1
    while (choice <= 0 or choice >= 4):
        print("Search Menu: \n1. Book Title\n2. Book Author\n3. Exit")
        choice = int(input("\nEnter your choice: "))
        if (choice == 1):
            book_name = str(input("\nEnter the book title: "))
            index = get_book_title(book_name)
        elif (choice == 2):
            book_author = str(input("\nEnter the author: "))
            index = get_book_author(book_author)
        elif (choice == 3):
            print("\nExiting...")
        else:
            print("\nIncorrect choice, please try again.\n")
    return index

In [62]:
def make_recommendation(n_recommendations = 5):
    book_id = search_book()
    if (book_id == -1):
        print("Book Not Found.")
        print("Done.")
        return -1, -1
    else: 
        corr_book_recs = correlation_mat[book_id-1]

        corr_dataframe = pd.DataFrame({'book_id': list(range(1, 10001)), 'correlation': corr_book_recs})
        final_dataframe = pd.merge(corr_dataframe, books_key, on='book_id')

        final_dataframe.sort_values(by='correlation', ascending=False, inplace=True)
        final_dataframe = final_dataframe.iloc[1:]
        final_dataframe.reset_index(inplace=True)
        final_dataframe.drop('index', axis=1, inplace=True)

        book_recs = final_dataframe.copy()
        book_recs = book_recs.iloc[:n_recommendations]
        book_recs.reset_index(inplace=True)
        book_recs.drop('index', axis=1, inplace=True)
        
        print("Done.")
        return book_recs, final_dataframe

In [67]:
recs, df = make_recommendation()

Search Menu: 
1. Book Title
2. Book Author
3. Exit

Enter your choice: 2

Enter the author: sarah j. maas
393: Throne of Glass (Throne of Glass, #1) by Sarah J. Maas
752: Crown of Midnight (Throne of Glass, #2) by Sarah J. Maas
842: A Court of Thorns and Roses (A Court of Thorns and Roses, #1) by Sarah J. Maas
1022: Heir of Fire (Throne of Glass, #3) by Sarah J. Maas
1264: Queen of Shadows (Throne of Glass, #4) by Sarah J. Maas
1308: A Court of Mist and Fury (A Court of Thorns and Roses, #2) by Sarah J. Maas
2167: Empire of Storms (Throne of Glass, #5) by Sarah J. Maas
2629: The Assassin's Blade (Throne of Glass, #0.1-0.5) by Sarah J. Maas
7373: A Court of Wings and Ruin (A Court of Thorns and Roses, #3) by Sarah J. Maas
7391: The Assassin and the Pirate Lord (Throne of Glass, #0.1) by Sarah J. Maas
7621: The Assassin and the Desert (Throne of Glass, #0.3) by Sarah J. Maas
8363: The Assassin and the Empire (Throne of Glass, #0.5) by Sarah J. Maas
8668: The Assassin and the Underworld (

In [68]:
recs

,book_id,correlation,authors,title
0,1264,0.999784,Sarah J. Maas,"Queen of Shadows (Throne of Glass, #4)"
1,1022,0.998848,Sarah J. Maas,"Heir of Fire (Throne of Glass, #3)"
2,7373,0.998272,Sarah J. Maas,A Court of Wings and Ruin (A Court of Thorns a...
3,752,0.997853,Sarah J. Maas,"Crown of Midnight (Throne of Glass, #2)"
4,5793,0.996108,Mary E. Pearson,"The Heart of Betrayal (The Remnant Chronicles,..."


In [73]:
recs, df = make_recommendation()

Search Menu: 
1. Book Title
2. Book Author
3. Exit

Enter your choice: 2

Enter the author: v.e. schwab
2116: A Darker Shade of Magic (Shades of Magic, #1) by V.E. Schwab, Victoria Schwab
4386: A Gathering of Shadows (Shades of Magic, #2) by V.E. Schwab, Victoria Schwab
5442: Vicious (Villains, #1) by V.E. Schwab, Victoria Schwab

Enter the book index (enter -1 if not found): 5442
Done.


In [74]:
recs

,book_id,correlation,authors,title
0,4386,0.977960,"V.E. Schwab, Victoria Schwab","A Gathering of Shadows (Shades of Magic, #2)"
1,2116,0.977404,"V.E. Schwab, Victoria Schwab","A Darker Shade of Magic (Shades of Magic, #1)"
2,1728,0.950569,Naomi Novik,Uprooted
3,6008,0.943616,C.S. Pacat,"Captive Prince (Captive Prince, #1)"
4,9964,0.940139,Samantha Shannon,"The Mime Order (The Bone Season, #2)"


In [76]:
recs, df = make_recommendation()

Search Menu: 
1. Book Title
2. Book Author
3. Exit

Enter your choice: 1

Enter the book title: uprooted
1728: Uprooted

Enter the book index (enter -1 if not found): 1728
Done.


In [77]:
recs

,book_id,correlation,authors,title
0,2116,0.988652,"V.E. Schwab, Victoria Schwab","A Darker Shade of Magic (Shades of Magic, #1)"
1,8365,0.977360,Genevieve Cogman,"The Invisible Library (The Invisible Library, #1)"
2,4386,0.967995,"V.E. Schwab, Victoria Schwab","A Gathering of Shadows (Shades of Magic, #2)"
3,6538,0.962458,Charlie N. Holmberg,The Master Magician (The Paper Magician Trilog...
4,4808,0.961905,Charlie N. Holmberg,The Glass Magician (The Paper Magician Trilogy...


In [78]:
recs, df = make_recommendation()

Search Menu: 
1. Book Title
2. Book Author
3. Exit

Enter your choice: 2

Enter the author: jk rowling
2: Harry Potter and the Sorcerer's Stone (Harry Potter, #1) by J.K. Rowling, Mary GrandPré
18: Harry Potter and the Prisoner of Azkaban (Harry Potter, #3) by J.K. Rowling, Mary GrandPré, Rufus Beck
21: Harry Potter and the Order of the Phoenix (Harry Potter, #5) by J.K. Rowling, Mary GrandPré
23: Harry Potter and the Chamber of Secrets (Harry Potter, #2) by J.K. Rowling, Mary GrandPré
24: Harry Potter and the Goblet of Fire (Harry Potter, #4) by J.K. Rowling, Mary GrandPré
25: Harry Potter and the Deathly Hallows (Harry Potter, #7) by J.K. Rowling, Mary GrandPré
27: Harry Potter and the Half-Blood Prince (Harry Potter, #6) by J.K. Rowling, Mary GrandPré
342: The Casual Vacancy by J.K. Rowling
399: The Tales of Beedle the Bard by J.K. Rowling
422: Harry Potter Boxset (Harry Potter, #1-7) by J.K. Rowling
2101: The Harry Potter Collection 1-4 (Harry Potter, #1-4) by J.K. Rowling, Mary Gr

In [79]:
recs

,book_id,correlation,authors,title
0,7443,0.998582,"J.K. Rowling, MinaLima","Short Stories from Hogwarts of Power, Politics..."
1,6428,0.995303,"J.K. Rowling, MinaLima","Short Stories from Hogwarts of Heroism, Hardsh..."
2,279,0.987585,"John Tiffany, Jack Thorne, J.K. Rowling",Harry Potter and the Cursed Child - Parts One ...
3,7550,0.974061,Sarah Andersen,"Adulthood Is a Myth (Sarah's Scribbles, #1)"
4,4641,0.967813,J.K. Rowling,Fantastic Beasts and Where to Find Them: The O...


In [80]:
recs, df = make_recommendation()

Search Menu: 
1. Book Title
2. Book Author
3. Exit

Enter your choice: 2

Enter the author: andy weir
141: The Martian by Andy Weir
9850: The Egg by Andy Weir

Enter the book index (enter -1 if not found): 141
Done.


In [81]:
recs

,book_id,correlation,authors,title
0,5921,0.973264,Sylvain Neuvel,"Sleeping Giants (Themis Files, #1)"
1,2832,0.970051,Claire North,The First Fifteen Lives of Harry August
2,4321,0.964163,Max Barry,Lexicon
3,9546,0.959920,Christopher Moore,"Secondhand Souls (Grim Reaper, #2)"
4,1736,0.959522,Blake Crouch,Dark Matter


In [82]:
recs, df = make_recommendation()

Search Menu: 
1. Book Title
2. Book Author
3. Exit

Enter your choice: 2

Enter the author: carl sagan
1003: Contact by Carl Sagan
1235: Cosmos by Carl Sagan
2226: The Demon-Haunted World: Science as a Candle in the Dark by Carl Sagan, Ann Druyan
4661: Pale Blue Dot: A Vision of the Human Future in Space by Carl Sagan, Ann Druyan
6202: The Portable Atheist: Essential Readings for the Nonbeliever by Christopher Hitchens, Titus Lucretius Carus, Omar Khayyám, Thomas Hobbes, Baruch Spinoza, David Hume, James Boswell, Percy Bysshe Shelley, George Eliot, Charles Darwin, Leslie Stephen, Anatole France, Mark Twain, Joseph Conrad, Thomas Hardy, Emma Goldman, H.P. Lovecraft, Carl Van Doren, H.L. Mencken, Sigmund Freud, Albert Einstein, George Orwell, John Betjeman, Chapman Cohen, Bertrand Russell, Philip Larkin, Martin Gardner, Carl Sagan, John Updike, John Leslie Mackie, Michael Shermer, A.J. Ayer, Daniel C. Dennett, Charles Templeton, Richard Dawkins, Victor J. Stenger, Elizabeth S.  Anderson,

In [83]:
recs

,book_id,correlation,authors,title
0,439,0.983926,"Stephen Hawking, Cao Chi, Phạm Văn Thiều",A Brief History of Time
1,7142,0.982031,John Gribbin,In Search of Schrödinger's Cat: Quantum Physic...
2,1224,0.980643,Brian Greene,"The Elegant Universe: Superstrings, Hidden Dim..."
3,8623,0.980253,"Carl Sagan, Ann Druyan",Billions & Billions: Thoughts on Life and Deat...
4,6857,0.977801,Carl Sagan,Dragons of Eden: Speculations on the Evolution...


In [84]:
recs, df = make_recommendation()

Search Menu: 
1. Book Title
2. Book Author
3. Exit

Enter your choice: 1

Enter the book title: the giver
36: The Giver (The Giver, #1)
856: Gathering Blue (The Giver, #2)
1001: The Dinner
1174: The Girls
1290: Messenger (The Giver, #3)
1903: Son (The Giver, #4)
2932: The Diviners (The Diviners, #1)
3367: The Winner
3761: The Dirt: Confessions of the World's Most Notorious Rock Band
4602: The Lover
4615: The Liar
4739: Scott Pilgrim, Volume 5: Scott Pilgrim Vs. the Universe
4908: The Game: Penetrating the Secret Society of Pickup Artists
5617: The Girls
6327: The Nine: Inside the Secret World of the Supreme Court
6394: The Fever
6506: The Gift
6755: The River (Brian's Saga, #2)
6812: On the Edge (The Edge, #1)
6995: The Peacegiver: How Christ Offers to Heal Our Hearts and Homes
7684: The Gift
7868: The Firm
8106: The Invaders (Brotherband Chronicles, #2)
8583: The Sister
8951: The Ear, the Eye, and the Arm
9544: The Wave
9714: The River Why
9996: Bayou Moon (The Edge, #2)

Enter the bo

In [85]:
recs

,book_id,correlation,authors,title
0,90,0.953180,S.E. Hinton,The Outsiders
1,2148,0.939438,Walter Dean Myers,Monster
2,77,0.938277,"Louis Sachar, Louis Sachar","Holes (Holes, #1)"
3,3169,0.926451,Richard Connell,The Most Dangerous Game
4,234,0.925864,Laurie Halse Anderson,Speak


In [86]:
recs, df = make_recommendation()

Search Menu: 
1. Book Title
2. Book Author
3. Exit

Enter your choice: 1

Enter the book title: dune
126: Dune (Dune Chronicles #1)
2817: Chapterhouse: Dune (Dune Chronicles #6)
7292: Dude, Where's My Country?

Enter the book index (enter -1 if not found): 126
Done.


In [87]:
recs

,book_id,correlation,authors,title
0,371,0.980633,Robert A. Heinlein,Stranger in a Strange Land
1,276,0.977315,Isaac Asimov,Foundation (Foundation #1)
2,432,0.971184,Isaac Asimov,"I, Robot (Robot #0.1)"
3,1042,0.970826,Douglas Adams,Dirk Gently's Holistic Detective Agency (Dirk ...
4,3358,0.968714,Douglas Adams,The Hitchhiker's Guide to the Galaxy: A Trilog...


In [89]:
recs, df = make_recommendation()

Search Menu: 
1. Book Title
2. Book Author
3. Exit

Enter your choice: 1

Enter the book title: christine
578: Christine
2727: A History of God: The 4,000-Year Quest of Judaism, Christianity, and Islam
3056: Christy
6972: The Cross of Lead (Crispin, #1)

Enter the book index (enter -1 if not found): 578
Done.


In [90]:
recs

,book_id,correlation,authors,title
0,556,0.999342,Stephen King,Cujo
1,675,0.999308,Stephen King,Firestarter
2,705,0.998882,"Richard Bachman, Stephen King",Thinner
3,670,0.998725,Stephen King,The Dead Zone
4,553,0.998593,Stephen King,Needful Things


In [91]:
recs, df = make_recommendation()

Search Menu: 
1. Book Title
2. Book Author
3. Exit

Enter your choice: 1

Enter the book title: an absolutely remarkable thing

Enter the book index (enter -1 if not found): -1
Book Not Found.
Done.


In [55]:
df = make_recommendation('the Hunger games')

1: The Hunger Games (The Hunger Games, #1)
17: Catching Fire (The Hunger Games, #2)
20: Mockingjay (The Hunger Games, #3)
507: The Hunger Games Trilogy Boxset (The Hunger Games, #1-3)
717: The Hunger Games: Official Illustrated Movie Companion
9709: The Hunger Pains: A Parody

Enter the book index (enter -1 if not found): 1


In [57]:
df.head(10)

,book_id,correlation,title
0,1,1.000000,"The Hunger Games (The Hunger Games, #1)"
1,507,0.932902,The Hunger Games Trilogy Boxset (The Hunger Ga...
2,17,0.898791,"Catching Fire (The Hunger Games, #2)"
3,9252,0.897001,Wicked - Piano/Vocal Arrangement
4,20,0.895943,"Mockingjay (The Hunger Games, #3)"
5,73,0.878247,"The Host (The Host, #1)"
6,12,0.868352,"Divergent (Divergent, #1)"
7,3,0.867923,"Twilight (Twilight, #1)"
8,6969,0.849001,"The Mind Readers (Mind Readers, #1)"
9,2880,0.848214,"Divergent Series Complete Box Set (Divergent, ..."


In [58]:
df2 = make_recommendation('twilight')

3: Twilight (Twilight, #1)
220: Twilight: The Complete Illustrated Movie Companion
2697: Firelight (Firelight, #1)
3075: Twilight: The Graphic Novel, Vol. 1 (Twilight: The Graphic Novel, #1)
4234: Twilight (The Mediator, #6)
5245: Twilight: The Graphic Novel, Vol. 2  (Twilight: The Graphic Novel, #2)
8251: Starlight (Warriors: The New Prophecy, #4)
8354: Twilight (Warriors: The New Prophecy, #5)
8389: Flight

Enter the book index (enter -1 if not found): 3


In [59]:
df2.head(10)

,book_id,correlation,title
0,3,1.000000,"Twilight (Twilight, #1)"
1,1735,0.938737,"Size 12 Is Not Fat (Heather Wells, #1)"
2,220,0.931060,Twilight: The Complete Illustrated Movie Compa...
3,5375,0.929392,The Secret Diaries of Miss Miranda Cheever (Be...
4,73,0.924587,"The Host (The Host, #1)"
5,2069,0.921294,"Queen of Babble (Queen of Babble, #1)"
6,4218,0.920063,"Queen of Babble Gets Hitched (Queen of Babble,..."
7,2238,0.913923,"The Duke and I (Bridgertons, #1)"
8,2075,0.913080,"The Boy Next Door (Boy, #1)"
9,1109,0.910998,New Moon: The Complete Illustrated Movie Compa...


In [11]:
# The rest of the code is testing the recommendation model 
# > 0.95 correlation
recs, corr = make_recommendation('the Hunger games')

1: The Hunger Games (The Hunger Games, #1)
17: Catching Fire (The Hunger Games, #2)
20: Mockingjay (The Hunger Games, #3)
507: The Hunger Games Trilogy Boxset (The Hunger Games, #1-3)
717: The Hunger Games: Official Illustrated Movie Companion
9709: The Hunger Pains: A Parody

Enter the book index (enter -1 if not found): 1

Recommendations:


In [27]:
# > 0.95 correlation
recs, corr = make_recommendation('twilight')

3: Twilight (Twilight, #1)
220: Twilight: The Complete Illustrated Movie Companion
2697: Firelight (Firelight, #1)
3075: Twilight: The Graphic Novel, Vol. 1 (Twilight: The Graphic Novel, #1)
4234: Twilight (The Mediator, #6)
5245: Twilight: The Graphic Novel, Vol. 2  (Twilight: The Graphic Novel, #2)
8251: Starlight (Warriors: The New Prophecy, #4)
8354: Twilight (Warriors: The New Prophecy, #5)
8389: Flight

Enter the book index (enter -1 if not found): 3

Recommendations:


In [34]:
# > 0.85 correlation
recs, corr = make_recommendation('the hunger games')

1: The Hunger Games (The Hunger Games, #1)
17: Catching Fire (The Hunger Games, #2)
20: Mockingjay (The Hunger Games, #3)
717: The Hunger Games: Official Illustrated Movie Companion

Enter the book index (enter -1 if not found): 1

Recommendations:
Twilight (Twilight, #1)
Divergent (Divergent, #1)
Catching Fire (The Hunger Games, #2)
Mockingjay (The Hunger Games, #3)
The Host (The Host, #1)
The Hunger Games Trilogy Boxset (The Hunger Games, #1-3)
Wicked - Piano/Vocal Arrangement


In [35]:
# > 0.85 correlation
recs, corr = make_recommendation('twilight')

3: Twilight (Twilight, #1)
220: Twilight: The Complete Illustrated Movie Companion
2697: Firelight (Firelight, #1)
3075: Twilight: The Graphic Novel, Vol. 1 (Twilight: The Graphic Novel, #1)
4234: Twilight (The Mediator, #6)
5245: Twilight: The Graphic Novel, Vol. 2  (Twilight: The Graphic Novel, #2)
8251: Starlight (Warriors: The New Prophecy, #4)
8354: Twilight (Warriors: The New Prophecy, #5)
8389: Flight

Enter the book index (enter -1 if not found): 3

Recommendations:
The Hunger Games (The Hunger Games, #1)
The Notebook (The Notebook, #1)
New Moon (Twilight, #2)
Eclipse (Twilight, #3)
Breaking Dawn (Twilight, #4)
The Host (The Host, #1)
A Walk to Remember
Confessions of a Shopaholic (Shopaholic, #1)
The Last Song
Twilight: The Complete Illustrated Movie Companion
P.S. I Love You
Can You Keep a Secret?
The Undomestic Goddess
Remember Me?
The Twilight Saga Breaking Dawn Part 1: The Official Illustrated Movie Companion (The Twilight Saga: The Official Illustrated Movie Companion, #4

In [12]:
recs, corr = make_recommendation('the Hunger games')

1: The Hunger Games (The Hunger Games, #1)
17: Catching Fire (The Hunger Games, #2)
20: Mockingjay (The Hunger Games, #3)
507: The Hunger Games Trilogy Boxset (The Hunger Games, #1-3)
717: The Hunger Games: Official Illustrated Movie Companion
9709: The Hunger Pains: A Parody

Enter the book index (enter -1 if not found): 17

Recommendations:
Mockingjay (The Hunger Games, #3)
Divergent Series Complete Box Set (Divergent, #1-3)


In [13]:
recs, corr = make_recommendation('catcher in the rye')

8: The Catcher in the Rye
1404: Catherine the Great: Portrait of a Woman

Enter the book index (enter -1 if not found): 8

Recommendations:
The Great Gatsby
The Catcher in the Rye
1984
Animal Farm
Lord of the Flies
Of Mice and Men
One Flew Over the Cuckoo's Nest
A Streetcar Named Desire
The Story of Art
Who's Afraid of Virginia Woolf?
Angels in America, Part Two: Perestroika
Equus
Art in Theory 1900 - 2000: An Anthology of Changing Ideas


In [14]:
recs, corr = make_recommendation('harry potter and the chamber of secrets')

23: Harry Potter and the Chamber of Secrets (Harry Potter, #2)
24: Harry Potter and the Goblet of Fire (Harry Potter, #4)
3054: Harry Potter and the Chamber of Secrets: Sheet Music for Flute with C.D

Enter the book index (enter -1 if not found): 23

Recommendations:
Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)
Harry Potter and the Order of the Phoenix (Harry Potter, #5)
Harry Potter and the Goblet of Fire (Harry Potter, #4)
Harry Potter and the Deathly Hallows (Harry Potter, #7)
Harry Potter and the Half-Blood Prince (Harry Potter, #6)


In [15]:
recs, corr = make_recommendation('the fault in our stars')

6: The Fault in Our Stars

Enter the book index (enter -1 if not found): 6

Recommendations:
Eleanor & Park
We Were Liars
Attachments
The Spectacular Now
The Lover's Dictionary
I Wrote This For You
If You Find Me


In [16]:
recs, corr = make_recommendation('to all the boys i loved before')

827: To All the Boys I've Loved Before (To All the Boys I've Loved Before, #1)
2112: P.S. I Still Love You (To All the Boys I've Loved Before, #2)

Enter the book index (enter -1 if not found): 827

Recommendations:
If I Stay (If I Stay, #1)
The Selection (The Selection, #1)
Fangirl
Anna and the French Kiss (Anna and the French Kiss, #1)
The Elite (The Selection, #2)
Before I Fall
The 5th Wave (The 5th Wave, #1)
Where She Went (If I Stay, #2)
The One (The Selection, #3)
Pandemonium (Delirium, #2)
Red Queen (Red Queen, #1)
The Unbecoming of Mara Dyer (Mara Dyer, #1)
Lola and the Boy Next Door (Anna and the French Kiss, #2)
Four: A Divergent Story Collection (Divergent, #0.1 - 0.4)
Reached (Matched, #3)
The Heir (The Selection, #4)
Requiem (Delirium, #3)
The Statistical Probability of Love at First Sight
The Infinite Sea (The 5th Wave, #2)
Let It Snow: Three Holiday Romances
Amy & Roger's Epic Detour
The Sky Is Everywhere
Just One Day (Just One Day, #1)
Dash & Lily's Book of Dares (Dash 

In [17]:
recs

[149,
 206,
 324,
 363,
 423,
 435,
 452,
 475,
 506,
 525,
 536,
 995,
 1004,
 1045,
 1054,
 1062,
 1118,
 1297,
 1366,
 1575,
 1720,
 1767,
 1849,
 1919,
 1951,
 2104,
 2112,
 2161,
 2179,
 2227,
 2532,
 2726,
 2734,
 2799,
 2838,
 2908,
 2962,
 3039,
 3328,
 3372,
 3389,
 3655,
 3846,
 3879,
 4021,
 4024,
 4033,
 4135,
 4143,
 4165,
 4195,
 4350,
 5079,
 5105,
 5184,
 5279,
 5308,
 5597,
 5712,
 5842,
 5974,
 6434,
 6466,
 6470,
 6478,
 6515,
 6805,
 6881,
 6989,
 7013,
 7190,
 7355,
 7362,
 7400,
 7412,
 7535,
 7653,
 7672,
 7802,
 7915,
 8041,
 8043,
 8095,
 8116,
 8119,
 8139,
 8268,
 8554,
 8573,
 8574,
 8869,
 8935,
 9009,
 9634,
 9725,
 9880,
 9893]

In [18]:
corr[4020]

0.9835507070924772

In [19]:
recs, corr = make_recommendation('the martian')

141: The Martian

Enter the book index (enter -1 if not found): 141

Recommendations:
The Girl with All the Gifts
Dark Matter
The First Fifteen Lives of Harry August
The Revenant
The City of Mirrors (The Passage, #3)
Lexicon
Brilliance (Brilliance Saga, #1)
Sleeping Giants (Themis Files, #1)
The Subtle Art of Not Giving a F*ck: A Counterintuitive Approach to Living a Good Life
Moriarty (Sherlock Holmes #2)
Secondhand Souls (Grim Reaper, #2)


In [20]:
recs, corr = make_recommendation('the great gatsby')

5: The Great Gatsby

Enter the book index (enter -1 if not found): 5

Recommendations:
To Kill a Mockingbird
The Great Gatsby
The Catcher in the Rye
Animal Farm
Lord of the Flies
Of Mice and Men
A Streetcar Named Desire
Who's Afraid of Virginia Woolf?
Cat on a Hot Tin Roof
Hedda Gabler
A Rose for Emily and Other Stories
All My Sons


In [21]:
recs, corr = make_recommendation('vampire diaries')

1387: Origins (The Vampire Diaries: Stefan's Diaries, #1)
3083: Nightfall (The Vampire Diaries: The Return, #1)
3226: The Ripper (The Vampire Diaries: Stefan's Diaries, #4)
3486: Shadow Souls (The Vampire Diaries: The Return, #2)
5759: Midnight (The Vampire Diaries: The Return, #3)
8266: Bloodlust (The Vampire Diaries: Stefan's Diaries, #2)

Enter the book index (enter -1 if not found): 1387

Recommendations:
Twilight: The Complete Illustrated Movie Companion
Marked (House of Night, #1)
The Twilight Saga Breaking Dawn Part 1: The Official Illustrated Movie Companion (The Twilight Saga: The Official Illustrated Movie Companion, #4)
The Hunger Games: Official Illustrated Movie Companion
Eclipse: The Complete Illustrated Movie Companion (The Twilight Saga: The Official Illustrated Movie Companion, #3)
The Twilight Saga Complete Collection  (Twilight, #1-4 + 3.5)
Vampire Academy Box Set (Vampire Academy, #1-4)
The Ripper (The Vampire Diaries: Stefan's Diaries, #4)
The Twilight Saga: The Of

In [22]:
recs, corr = make_recommendation('cinders')

252: Cinder (The Lunar Chronicles, #1)
4100: Tinkers

Enter the book index (enter -1 if not found): 252

Recommendations:
The Maze Runner (Maze Runner, #1)
Matched (Matched, #1)
The Selection (The Selection, #1)
The Scorch Trials (Maze Runner, #2)
Delirium (Delirium, #1)
Legend (Legend, #1)
I Am Number Four (Lorien Legacies, #1)
The Death Cure (Maze Runner, #3)
Throne of Glass (Throne of Glass, #1)
The Elite (The Selection, #2)
The 5th Wave (The 5th Wave, #1)
Clockwork Princess (The Infernal Devices, #3)
The One (The Selection, #3)
Pandemonium (Delirium, #2)
Daughter of Smoke & Bone (Daughter of Smoke & Bone, #1)
Red Queen (Red Queen, #1)
Scarlet (The Lunar Chronicles, #2)
City of Heavenly Fire (The Mortal Instruments, #6)
Shatter Me (Shatter Me, #1)
Crossed (Matched, #2)
Gone (Gone, #1)
Unwind (Unwind, #1)
Prodigy (Legend, #2)
Crown of Midnight (Throne of Glass, #2)
Cress (The Lunar Chronicles, #3)
A Court of Thorns and Roses (A Court of Thorns and Roses, #1)
The Raven Boys (The Raven

In [23]:
recs, corr = make_recommendation('the selection')

206: The Selection (The Selection, #1)
423: The Elite (The Selection, #2)
506: The One (The Selection, #3)
1062: The Heir (The Selection, #4)
2179: The Crown (The Selection, #5)

Enter the book index (enter -1 if not found): 206

Recommendations:
Matched (Matched, #1)
Cinder (The Lunar Chronicles, #1)
The Scorch Trials (Maze Runner, #2)
Delirium (Delirium, #1)
Legend (Legend, #1)
Anna and the French Kiss (Anna and the French Kiss, #1)
Throne of Glass (Throne of Glass, #1)
The Elite (The Selection, #2)
Before I Fall
The 5th Wave (The 5th Wave, #1)
Clockwork Princess (The Infernal Devices, #3)
The One (The Selection, #3)
Pandemonium (Delirium, #2)
Daughter of Smoke & Bone (Daughter of Smoke & Bone, #1)
Red Queen (Red Queen, #1)
Scarlet (The Lunar Chronicles, #2)
City of Heavenly Fire (The Mortal Instruments, #6)
Shatter Me (Shatter Me, #1)
Crossed (Matched, #2)
Prodigy (Legend, #2)
Crown of Midnight (Throne of Glass, #2)
Cress (The Lunar Chronicles, #3)
To All the Boys I've Loved Before 

In [24]:
recs, corr = make_recommendation('six of crows')

1668: Six of Crows (Six of Crows, #1)

Enter the book index (enter -1 if not found): 1668

Recommendations:
Cinder (The Lunar Chronicles, #1)
The Scorch Trials (Maze Runner, #2)
The Death Cure (Maze Runner, #3)
The 5th Wave (The 5th Wave, #1)
Red Queen (Red Queen, #1)
Scarlet (The Lunar Chronicles, #2)
Prodigy (Legend, #2)
Crown of Midnight (Throne of Glass, #2)
Cress (The Lunar Chronicles, #3)
A Court of Thorns and Roses (A Court of Thorns and Roses, #1)
The Raven Boys (The Raven Cycle, #1)
The Kill Order (Maze Runner, #0.5)
Shadow and Bone (Shadow and Bone, #1)
Champion (Legend, #3)
Heir of Fire (Throne of Glass, #3)
Queen of Shadows (Throne of Glass, #4)
A Court of Mist and Fury (A Court of Thorns and Roses, #2)
Winter (The Lunar Chronicles, #4)
The Infinite Sea (The 5th Wave, #2)
An Ember in the Ashes (An Ember in the Ashes, #1)
Days of Blood & Starlight (Daughter of Smoke & Bone, #2)
Carry On
Glass Sword (Red Queen, #2)
Siege and Storm (Shadow and Bone, #2)
The Dream Thieves (The 

In [25]:
recs, corr = make_recommendation('divergent')

12: Divergent (Divergent, #1)

Enter the book index (enter -1 if not found): 12

Recommendations:
Insurgent (Divergent, #2)
Allegiant (Divergent, #3)
Warm Bodies (Warm Bodies, #1)
The Transfer (Divergent, #0.1)
The World of Divergent: The Path to Allegiant (Divergent, #2.5)
The Initiate (Divergent, #0.2)
The Traitor (Divergent, #0.4)
The Son (Divergent, #0.3)


In [9]:
x = [1, 2, 3, 4, 5]
x[2:4]

[3, 4]

In [ ]:
# Original get_book (searching for a book from its title)
def get_book (book_name):
    for key, book in book_hashmap.items():
        temp = re.split('[(,.:)]', book)
        x = 0
        while x < len(temp):
            if SequenceMatcher(a=book_name, b=temp[x]).ratio() > 0.7:
                print(str(key) + ": " + book)
                x = len(temp)
            x = x + 1
    index = int(input("\nEnter the book index (enter -1 if not found): "))
    return index

In [ ]:
# original make recommendation (only searches by book title)
def make_recommendation(book_name, n_recommendations = 5):
    book_id = get_book(book_name)
    
    corr_book_recs = correlation_mat[book_id-1]
    # array_indices = list((corr_book_recs<1.0) and (corr_book_recs>0.9))
    corr_dataframe = pd.DataFrame({'book_id': list(range(1, 10001)), 'correlation': corr_book_recs})
    final_dataframe = pd.merge(corr_dataframe, books_key, on='book_id')

    final_dataframe.sort_values(by='correlation', ascending=False, inplace=True)
    final_dataframe = final_dataframe.iloc[1:]
    final_dataframe.reset_index(inplace=True)
    final_dataframe.drop('index', axis=1, inplace=True)

    book_recs = final_dataframe.copy()
    book_recs = book_recs.iloc[:n_recommendations]
    book_recs.reset_index(inplace=True)
    book_recs.drop('index', axis=1, inplace=True)
    #print("\nRecommendations:")
    #for x in range(0, len(corr_book_recs)):
        #if corr_book_recs[x] < 1.0 and corr_book_recs[x] > 0.85:
        #    print(hashmap[x+1])
        #    book_recs.append(x+1)
        #book_recs.append(hashmap[array_indices[x]])
    return book_recs, final_dataframe